In [194]:
%matplotlib ipympl
import os
import os.path as path
import pandas as pd
import h5py
import numpy as np
from faker import Factory
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly
from ipywidgets import fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from IPython.display import clear_output
from IPython.display import display
from ipywidgets import IntSlider, Output
import plotly.graph_objects as go
import plotly.express as plot_x

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, data_tools, variables, calibration, data_loadcrop, ion_selection, reconstruction, selectors_data
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.calibration_tools import dataset_path_qt
from pyccapt.calibration.leap_tools import ccapt_tools

In [184]:
# Disable showing logging in Jupyter notebook
import logging, sys
logging.disable(sys.maxsize)

In [68]:
button = widgets.Button(
    description='load dataset',
)

@button.on_click
def open_file_on_click(b):
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [167]:
tdc, det_diam, pulse_mode, flightPathLength, t0_d, max_tof = wd.dataset_tdc_selection()
display(tdc, pulse_mode, flightPathLength)

Dropdown(description='TDC model:', options=('surface_concept', 'roentdec'), value='surface_concept')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

$$\textbf{You can specify which dataset to use in below block}$$

In [195]:
tdc_model = tdc.value
pulse_mode_ini = pulse_mode.value
flight_path_length = flightPathLength.value

dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
dataset_name = os.path.splitext(dataset_name_with_extention)[0]


variables.init()
# variables.path = os.path.join(p, 'tests//data')
variables.result_path = os.path.dirname(dataset_main_path) + '/reconstruction/'
if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
        
filename = dataset_path

head, tail = os.path.split(filename)
figname = os.path.splitext(tail)[0]

data = data_tools.read_hdf5_through_pandas(filename)
# range_data = None

In [169]:
data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse (V),start_counter,t (ns),t_c (nm),x_det (cm),y_det (cm),pulse_pi,ion_pp,tof_c (ns)
0,0.0,0.0,0.0,27.352441,27.401002,5269.834473,5269.834473,54436.0,635.421132,0.0,-1.595020,-1.661878,0,1,0.0
1,0.0,0.0,0.0,27.228966,27.522766,5269.834473,5269.834473,54903.0,635.901192,0.0,0.646286,2.129878,467,1,0.0
2,0.0,0.0,0.0,27.086795,27.582728,5269.834473,5269.834473,55353.0,648.101574,0.0,-0.175102,3.155020,450,1,0.0
3,0.0,0.0,0.0,27.282802,26.913047,5269.834473,5269.834473,55831.0,636.010920,0.0,1.964327,-1.999347,478,1,0.0
4,0.0,0.0,0.0,27.132434,27.273478,5269.834473,5269.834473,56022.0,633.350016,0.0,1.381714,1.757388,191,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898626,0.0,0.0,0.0,248.242374,248.363850,5711.471680,5711.471680,37587.0,1716.831720,0.0,0.369306,-1.330776,83,1,0.0
1898627,0.0,0.0,0.0,27.040206,27.320343,5711.471680,5711.471680,38162.0,600.993972,0.0,0.423429,0.630367,575,1,0.0
1898628,0.0,0.0,0.0,27.665819,27.881308,5711.471680,5711.471680,38200.0,606.096324,0.0,0.560327,0.200571,38,1,0.0
1898629,0.0,0.0,0.0,27.019084,27.072135,5711.471680,5711.471680,38467.0,600.026994,0.0,0.926449,-0.633551,267,1,0.0


In [76]:
# exctract needed data from Pandas data frame as an numpy array
# dld_highVoltage = data['high_voltage (V)'].to_numpy()
# dld_x = data['x_det (cm)'].to_numpy()
# dld_y = data['y_det (cm)'].to_numpy()
# mc = data['mc_c (Da)'].to_numpy()

In [77]:
def mc_hist_plot(mc, bin_size, prominence, distance, percent, selector, plot, figname, lim_mc):
    peaks_ini, peaks_y_ini, peak_widths_p_ini = tools.hist_plot(mc[mc < lim_mc], bin_size, distance=distance, percent=percent, prominence=prominence, selector=selector, plot=plot, label='mc', fig_name=figname+'_ini')
    index_max_ini = np.argmax(peaks_y_ini)
    variables.max_peak = peaks_ini[index_max_ini]
    variables.peak = peaks_ini
    mrp = (peaks_ini[index_max_ini] / (peak_widths_p_ini[index_max_ini][2] - peak_widths_p_ini[index_max_ini][1]))
    print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
    for i in range(len(peaks_ini)):
        print('Peaks ', i, 'is at location and height: ({:.2f}, {:.2f})'.format(peaks_ini[i], peaks_y_ini[i]), 'peak window sides (half-maximum) are: ({:.2f}, {:.2f})'.format(peak_widths_p_ini[i][1], peak_widths_p_ini[i][2]))

interact_manual(mc_hist_plot, mc=fixed(mc), bin_size=widgets.FloatText(value=0.1)
               ,prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), lim_mc=widgets.IntText(value=150)
               ,percent=widgets.IntText(value=50), selector=fixed('rect'), plot=fixed(True), figname=fixed(figname));


interactive(children=(FloatText(value=0.1, description='bin_size'), IntText(value=50, description='prominence'…

In [78]:
element_selected = wd.density_field_selection()
display(element_selected)

Dropdown(description='Element', options=((1, 'H', 52.7, 0.0), (2, 'He', 28.7, 0.0), (3, 'Li', 46.3, 14.0), (4,…

In [79]:
avg_dens = element_selected.value[2]
field_evap = element_selected.value[3]
flight_path_length = flightPathLength_d.value 

In [81]:
# reconstruction_calculation_plot(dld_x, dld_y, dld_highVoltage, data, range_data, element_percentage, flight_path_length=110, kf=4, det_eff=0.7, icf=1.4, field_evap=19, avg_dens=60.2, rotary_fig_save=False, range_file_exist=True)

In [82]:
button_range = widgets.Button(
    description='load range',
)

@button_range.on_click
def open_file_on_click(b):
    global range_path
    range_path = dataset_path_qt.gui_fname().decode('ASCII')
button_range


Button(description='load range', style=ButtonStyle())

In [198]:
range_data = data_tools.read_hdf5_through_pandas(range_path)
range_data

,element,isotope,charge,mc_low,mc_up,color
0,H,1,1,1.100636,1.300752,#b2aa2d
1,Al,27,2,13.607863,13.908036,#e7e0d1
2,Al,27,1,26.815494,27.315783,#e7e0d1


In [205]:
def x_y_z_calculation_and_plot(data, range_data, element_percentage, kf, det_eff, icf, field_evap, avg_dens, rotary_fig_save, range_file_exist, selected_are, figname):
    dld_highVoltage = data['high_voltage (V)'].to_numpy()
    dld_x = data['x_det (cm)'].to_numpy()
    dld_y = data['y_det (cm)'].to_numpy()
    mc = data['mc_c (Da)'].to_numpy()
    flight_path_length = flightPathLength.value
    px, py, pz = reconstruction.atom_probe_recons_from_detector_Gault_et_al(dld_x, dld_y, dld_highVoltage,
                                                                            flight_path_length, kf, det_eff, icf,
                                                                            field_evap, avg_dens)
    #     px, py, pz = reconstruction.atom_probe_recons_Bas_et_al(dld_x, dld_y, dld_highVoltage, flight_path_length, kf, det_eff, icfe, field_evap, avg_dens)
    data['x (nm)'] = px
    data['y (nm)'] = py
    data['z (nm)'] = pz
    reconstruction.reconstruction_plot(data, range_data, element_percentage, rotary_fig_save, range_file_exist, selected_are, figname)

In [206]:
if range_data is not None:
    element_percentage = [0.01] * len(range_data['element'].tolist())
    element_percentage = str(element_percentage)
else:
    element_percentage = str([0.01])

interact_manual(x_y_z_calculation_and_plot, kf=widgets.FloatText(value=4), det_eff=widgets.FloatText(value=0.7), icf=widgets.FloatText(value=1.4),
                field_evap=widgets.FloatText(value=field_evap),
               avg_dens=widgets.FloatText(value=avg_dens), element_percentage=widgets.Textarea(value=element_percentage),
               rotary_fig_save=widgets.Dropdown(options=[('True', True), ('False', False)], value=False), data=fixed(data), range_data=fixed(range_data),
                range_file_exist=widgets.Dropdown(options=[('True', True), ('False', False)], value=True), selected_are=fixed(False), figname=figname);

C:\Users\mehrp\AppData\Local\Temp\ipykernel_13572\3446342754.py:7: DeprecationWarning:

on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').



interactive(children=(Textarea(value='[0.01, 0.01, 0.01]', description='element_percentage'), FloatText(value=…

In [274]:
data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse (V),start_counter,t (ns),t_c (nm),x_det (cm),y_det (cm),pulse_pi,ion_pp,tof_c (ns)
0,-13.660657,-14.233260,2.865681,27.352441,27.401002,5269.834473,5269.834473,54436.0,635.421132,0.0,-1.595020,-1.661878,0,1,0.0
1,5.545922,18.276956,2.682450,27.228966,27.522766,5269.834473,5269.834473,54903.0,635.901192,0.0,0.646286,2.129878,467,1,0.0
2,-1.462241,26.346922,5.217198,27.086795,27.582728,5269.834473,5269.834473,55353.0,648.101574,0.0,-0.175102,3.155020,450,1,0.0
3,16.591314,-16.887107,4.166498,27.282802,26.913047,5269.834473,5269.834473,55831.0,636.010920,0.0,1.964327,-1.999347,478,1,0.0
4,11.853949,15.076912,2.705165,27.132434,27.273478,5269.834473,5269.834473,56022.0,633.350016,0.0,1.381714,1.757388,191,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898626,3.493864,-12.589959,NaN,248.242374,248.363850,5711.471680,5711.471680,37587.0,1716.831720,0.0,0.369306,-1.330776,83,1,0.0
1898627,4.036500,6.009226,NaN,27.040206,27.320343,5711.471680,5711.471680,38162.0,600.993972,0.0,0.423429,0.630367,575,1,0.0
1898628,5.348383,1.914478,NaN,27.665819,27.881308,5711.471680,5711.471680,38200.0,606.096324,0.0,0.560327,0.200571,38,1,0.0
1898629,8.797205,-6.015958,NaN,27.019084,27.072135,5711.471680,5711.471680,38467.0,600.026994,0.0,0.926449,-0.633551,267,1,0.0


In [280]:
def projection(data, range_data, element_percentage, selected_area, x_or_y, figname):
    ax = plt.figure().add_subplot(111)
        
    phases = range_data['element'].tolist()
    colors = range_data['color'].tolist()
    mc_low = range_data['mc_low'].tolist()
    mc_up = range_data['mc_up'].tolist()
    charge = range_data['charge'].tolist()
    isotope = range_data['isotope'].tolist()
    mc = data['mc_c (Da)']
        
    
    element_percentage = element_percentage.replace('[', '')
    element_percentage = element_percentage.replace(']', '')
    element_percentage = element_percentage.split(',')
    
    for index, elemen in enumerate(phases):
        df_s = data.copy(deep=True)
        df_s = df_s[(df_s['mc_c (Da)'] > mc_low[index]) & (df_s['mc_c (Da)'] < mc_up[index])]
        df_s.reset_index(inplace=True, drop=True)
        remove_n = int(len(df_s) - (len(df_s) * float(element_percentage[index])))
        # print(len(df_s))
        drop_indices = np.random.choice(df_s.index, remove_n, replace=False)
        df_subset = df_s.drop(drop_indices)
        # print(len(df_subset))
        name_element = r'${}^{%s}%s^{%s+}$' % (isotope[index], phases[index], charge[index])
        if x_or_y == 'x':
            ax.scatter(df_subset['x (nm)'], df_subset['z (nm)'], s=2, label=name_element, color=colors[index])
        elif x_or_y == 'y':
            ax.scatter(df_subset['y (nm)'], df_subset['z (nm)'], s=2, label=name_element)

    if not selected_area:
        data_loadcrop.rectangle_box_selector(ax)
        plt.connect('key_press_event', selectors_data.toggle_selector)
    ax.xaxis.tick_top()
    ax.invert_yaxis()
    if x_or_y == 'x':
            ax.set_xlabel('X (nm)')
    elif x_or_y == 'y':
            ax.set_xlabel('Y (nm)')
    ax.xaxis.set_label_position('top')
    ax.set_ylabel('Z (nm)')
    plt.legend(loc='upper right')
    plt.savefig(variables.result_path + '\\projection_{fn}.png'.format(fn=figname))
    plt.show()

In [323]:
def heatmap(data, range_data, selected_area, element_percentage, save):
    ax = plt.figure().add_subplot(111)
        
    phases = range_data['element'].tolist()
    colors = range_data['color'].tolist()
    mc_low = range_data['mc_low'].tolist()
    mc_up = range_data['mc_up'].tolist()
    charge = range_data['charge'].tolist()
    isotope = range_data['isotope'].tolist()
    mc = data['mc_c (Da)']
    
    element_percentage = element_percentage.replace('[', '')
    element_percentage = element_percentage.replace(']', '')
    element_percentage = element_percentage.split(',')
    
    for index, elemen in enumerate(phases):
        df_s = data.copy(deep=True)
        df_s = df_s[(df_s['mc_c (Da)'] > mc_low[index]) & (df_s['mc_c (Da)'] < mc_up[index])]
        df_s.reset_index(inplace=True, drop=True)
        remove_n = int(len(df_s) - (len(df_s) * float(element_percentage[index])))
        # print(len(df_s))
        drop_indices = np.random.choice(df_s.index, remove_n, replace=False)
        df_subset = df_s.drop(drop_indices)
        # print(len(df_subset))
        name_element = r'${}^{%s}%s^{%s+}$' % (isotope[index], phases[index], charge[index])
        # Plot heat map and convert from cm to mm
        xx = df_subset['x_det (cm)'].to_numpy() * 10
        yy = df_subset['y_det (cm)'].to_numpy() * 10
        ax.scatter(xx, yy, s=2, label=name_element, color=colors[index])



    # set x-axis label
    ax.set_xlabel("x [mm]", color="red", fontsize=10)
    # set y-axis label
    ax.set_ylabel("y [mm]", color="red", fontsize=10)
    plt.title("Detector Heatmap")
    plt.legend(loc='upper right')

    if save:
        plt.savefig(variables.result_path + "heatmap.png", format="png", dpi=600)
        plt.savefig(variables.result_path + "heatmap.svg", format="svg", dpi=600)
    plt.show()

In [324]:

plot_3d_button = widgets.Button(
    description='plot 3D',
)
plot_heatmap_button = widgets.Button(
    description='plot heatmap',
)
plot_mc_button = widgets.Button(
    description='plot mc',
)
plot_projection_button = widgets.Button(
    description='plot projection',
)

clear_button = widgets.Button(
    description='Clear plots',
)

selected_area_p3=widgets.Dropdown(options=[('True', True), ('False', False)], description='Selected area')
range_file_exist_p3=widgets.Dropdown(options=[('True', True), ('False', False)], description='Range exist')
rotary_fig_save_p3=widgets.Dropdown(options=[('True', True), ('False', False)], description='Rotary save')
element_percentage_p3=widgets.Textarea(value=element_percentage, description='Element percentage')
@plot_3d_button.on_click
def plot_3d(b, data=data, range_data=range_data, figname=figname):
    with out:
        if not variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
            variables.selected_calculated = True
        reconstruction.reconstruction_plot(data, range_data, element_percentage_p3.value, rotary_fig_save_p3.value, range_file_exist_p3.value, selected_area_p3.value, figname)

selected_area_ph=widgets.Dropdown(options=[('True', True), ('False', False)], description='Selected area')
element_percentage_ph=widgets.Textarea(value=element_percentage, description='Element percentage')
@plot_heatmap_button.on_click
def plot_heatmap(b, data=data, range_data=range_data):
    with out:
        if not variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
            variables.selected_calculated = True
        data_f = data.copy(deep=True)
        if selected_area_ph.value:
            data_f = data_f[(data_f['x (nm)'] >  variables.selected_x1) & (data_f['x (nm)'] <  variables.selected_x2) & 
                            (data_f['y (nm)'] >  variables.selected_y1) & (data_f['y (nm)'] <  variables.selected_y2) & 
                            (data_f['z (nm)'] >  variables.selected_z1) & (data_f['z (nm)'] <  variables.selected_z2)]

            data_f.reset_index(inplace=True, drop=True)
        heatmap(data_f, range_data, selected_area_ph.value, element_percentage_ph.value, save=True)

selected_area_pm=widgets.Dropdown(options=[('True', True), ('False', False)], description='Selected area')
range_file_exist_pm=widgets.Dropdown(options=[('True', True), ('False', False)], description='Range exist')
bin_size_pm = widgets.FloatText(value=0.1, description='Bins size')
prominence_pm = widgets.IntText(value=50, description='Prominance')
distance_pm = widgets.IntText(value=100, description='Distance')
lim_mc_pm = widgets.IntText(value=150, description='Limit mc')
percent_pm = widgets.IntText(value=50, description='Percent MRP')
@plot_mc_button.on_click
def plot_mc(b, data=data, range_data=range_data):
    with out:
        if not variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
            variables.selected_calculated = True
        data_f = data.copy(deep=True)
        if selected_area_pm.value:
            data_f = data_f[(data_f['x (nm)'] >  variables.selected_x1) & (data_f['x (nm)'] <  variables.selected_x2) & 
                            (data_f['y (nm)'] >  variables.selected_y1) & (data_f['y (nm)'] <  variables.selected_y2) & 
                            (data_f['z (nm)'] >  variables.selected_z1) & (data_f['z (nm)'] <  variables.selected_z2)]

            data_f.reset_index(inplace=True, drop=True)
        mc = data_f['mc_c (Da)'].to_numpy()

        peaks_ini, peaks_y_ini, peak_widths_p_ini = tools.hist_plot(mc[mc < lim_mc_pm.value], range_data=range_data, ranging=range_file_exist_pm.value, bin=bin_size_pm.value, distance=distance_pm.value, percent=percent_pm.value,
                                                                    prominence=prominence_pm.value, selector=None, plot=True, label='mc', fig_name=figname+'_ini')
        index_max_ini = np.argmax(peaks_y_ini)
        mrp = (peaks_ini[index_max_ini] / (peak_widths_p_ini[index_max_ini][2] - peak_widths_p_ini[index_max_ini][1]))
        print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
        for i in range(len(peaks_ini)):
            print('Peaks ', i, 'is at location and height: ({:.2f}, {:.2f})'.format(peaks_ini[i], peaks_y_ini[i]), 'peak window sides (half-maximum) are: ({:.2f}, {:.2f})'.format(peak_widths_p_ini[i][1], peak_widths_p_ini[i][2]))

element_percentage_pp=widgets.Textarea(value=element_percentage, description='Element percentage')
selected_area_pp=widgets.Dropdown(options=[('True', True), ('False', False)], description='Selected area')
x_or_y_pp=widgets.Dropdown(options=['x', 'y'], value='x', description='X or Y')

@plot_projection_button.on_click
def plot_projection(b, data=data, range_data=range_data, figname=figname):
    with out:
        if not variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
            variables.selected_calculated = True
        data_f = data.copy(deep=True)
        if selected_area_pp.value:
            data_f = data_f[(data_f['x (nm)'] >  variables.selected_x1) & (data_f['x (nm)'] <  variables.selected_x2) & 
                            (data_f['y (nm)'] >  variables.selected_y1) & (data_f['y (nm)'] <  variables.selected_y2) & 
                            (data_f['z (nm)'] >  variables.selected_z1) & (data_f['z (nm)'] <  variables.selected_z2)]
        
            data_f.reset_index(inplace=True, drop=True)
        projection(data_f, range_data, element_percentage_pp.value, selected_area_pp.value, x_or_y_pp.value, figname)

@clear_button.on_click
def clear(b,):
    with out:
        clear_output(True)
        print('')


tab1 = VBox(children=[selected_area_pp, x_or_y_pp, element_percentage_pp, plot_projection_button, clear_button])
tab2 = VBox(children=[selected_area_p3, range_file_exist_p3, rotary_fig_save_p3, element_percentage_p3, plot_3d_button, clear_button])
tab3 = VBox(children=[selected_area_pm, range_file_exist_pm, bin_size_pm, prominence_pm, distance_pm, lim_mc_pm, percent_pm, plot_mc_button, clear_button])
tab4 = VBox(children=[selected_area_ph, element_percentage_ph, plot_heatmap_button, clear_button])
# tab2 = VBox(children=[VBox(children=[HBox(children=[plot_3d_button, selected_area_p3, range_file_exist_p3, rotary_fig_save_p3, element_percentage_p3]),
#                                      HBox(children=[plot_heatmap_button, selected_area_ph, element_percentage_ph]),
#                                      HBox(children=[plot_mc_button, selected_area_pm, range_file_exist_pm, bin_size_pm, prominence_pm, distance_pm, lim_mc_pm, percent_pm]), 
#                                      HBox(children=[plot_projection_button, selected_area_pp, x_or_y_pp, element_percentage_pp])]), clear_button])


tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, 'projection')
tab.set_title(1, '3d plot')
tab.set_title(2, 'mc plot')
tab.set_title(3, 'heatmap plot')



display(VBox(children=[tab]))
out = Output()
display(out)

Output()

In [28]:
# save the new data
name_save_file = variables.result_path + '//' + dataset_name + '.h5'
data_tools.store_df_to_hdf(name_save_file, data, 'df')

In [29]:
# save data as epos file
epos = ccapt_tools.ccapt_to_epos(data, path=variables.result_path, name=dataset_name + '.epos')
pos = ccapt_tools.ccapt_to_pos(data, path=variables.result_path, name=dataset_name + '.POS')

In [ ]:
name_save_file = variables.result_path + '//' + dataset_name + '.csv'
data.to_csv(name_save_file, index = False)
